In [1]:
from dotenv import dotenv_values
from datasets import load_dataset, Dataset
from utils.data_preprocessor import DataPreprocessor
from utils.evaluator import Evaluator
from config.finetuning import config
from utils.load_merged_model_tokenizer import load_mergedModel_tokenizer
from config import postprocessing
from utils.test_data_processor import TestDataProcessor
import pandas as pd


HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']

adapters_checkpoints_list = postprocessing.adapters_checkpoints_list
base_models_list = postprocessing.base_models_list
splits_list = postprocessing.splits_list
max_new_tokens_factor_list = postprocessing.max_new_tokens_factor_list
n_shots_inference_list = postprocessing.n_shots_inference_list
language = postprocessing.splits_list[0].split('.')[0]


merged_model, tokenizer = load_mergedModel_tokenizer(adapters_checkpoints_list[0], base_models_list[0])

dataset = load_dataset("ferrazzipietro/e3c-sentences", token=HF_TOKEN)
dataset = dataset[splits_list[0]]
preprocessor = DataPreprocessor()
dataset = preprocessor.preprocess_data_one_layer(dataset)
_, val_data, _ = preprocessor.split_layer_into_train_val_test_(dataset, splits_list[0])


/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'load_model_tokenizer' from 'utils.load_merged_model_tokenizer' (/home/pferrazzi/mistral_finetuning/utils/load_merged_model_tokenizer.py)

In [ ]:
for max_new_tokens_factor in max_new_tokens_factor_list:
    for n_shots_inference in n_shots_inference_list:
        postprocessor = TestDataProcessor(test_data=val_data, preprocessor=preprocessor, 
                                          n_shots_inference=n_shots_inference, 
                                          language=language, 
                                          tokenizer=tokenizer)
        postprocessor.add_inference_prompt_column()
        postprocessor.add_ground_truth_column()
        postprocessor.add_responses_column(model=merged_model, 
                                           tokenizer=tokenizer, 
                                           batch_size=24, 
                                           max_new_tokens_factor=max_new_tokens_factor)
        postprocessor.test_data.to_csv(f"data/test_data_processed/{language}_nShots{n_shots_inference}_maxNewTokensFactor{max_new_tokens_factor}.csv", index=False)


### ONE RUN

In [33]:
from_backup_file = False
if from_backup_file:
    tmp_data = pd.read_csv(f"data/test_data_processed/en_nShots{n_shots_inference}_maxNewTokensFactor{max_new_tokens_factor}.csv")
    tmp_data = Dataset.from_pandas(tmp_data)
    postprocessor = TestDataProcessor(test_data=tmp_data, preprocessor=preprocessor, n_shots_inference=n_shots_inference, language=splits[0].split('.')[0], tokenizer=tokenizer)

if not from_backup_file:
    postprocessor = TestDataProcessor(test_data=val_data, preprocessor=preprocessor, n_shots_inference=n_shots_inference, language=splits[0].split('.')[0], tokenizer=tokenizer)
    postprocessor.add_inference_prompt_column()
    postprocessor.add_ground_truth_column()
    postprocessor.add_responses_column(model=merged_model, tokenizer=tokenizer, batch_size=24, max_new_tokens_factor=max_new_tokens_factor)
    postprocessor.test_data.to_csv(f"data/test_data_processed/{splits[0].split('.')[0]}_nShots{n_shots_inference}_maxNewTokensFactor{max_new_tokens_factor}.csv", index=False)

generating responses:   4%|▎         | 24/681 [00:46<21:16,  1.94s/it]